In [116]:
import requests
from transformers import BertTokenizer
import json
from dataclasses import dataclass
from time import time

In [ ]:
import os, sys
from pathlib import Path
FILE = Path("").resolve()
ROOT = FILE.parents[0].parents[0].parents[0]  # YOLOv5 root directory
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

In [109]:
from utils import timing

In [53]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text = "Paris is the [MASK] of France."
encoded_input = tokenizer(text, return_tensors='np')

In [71]:

{k:v.tolist() for k, v in encoded_input.items()}

{'input_ids': [[101, 3000, 2003, 1996, 103, 1997, 2605, 1012, 102]],
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [112]:
MODEL_NAME = "bert_base_uncased"
PORT = 8501
URL = f"http://localhost:{PORT}/v1/models/{MODEL_NAME}"

def columnar_inputs(inputs):
    # inputs should be a dictionary of ndarrays
    columnar_data = {k:v.tolist() for k, v in inputs.items()}
    return(columnar_data)

def model_service_metadata(url=URL):
    _url = url+"/metadata"
    r = requests.get(_url)
    return(r.json())

@timing
def model_serv_one_pred(inputs=encoded_input, url=URL):
    #POST http://host:port/v1/models/${MODEL_NAME}[/versions/${VERSION}|/labels/${LABEL}]:predict
    data = columnar_inputs(inputs)
    _url = url+":predict"
    data = json.dumps({"inputs":data})
    r = requests.post(_url,data=data)
    return(r.json())



def model_serv_batch_pred():
    return("TK")



In [117]:
# Bert Vanilla Tensorflow Benchmarking 1000 trials
NUM_REQUESTS = 1000
i = 0
results = []
while i<NUM_REQUESTS:
    ts = time()
    res = model_serv_one_pred() # Call TF container
    te = time()
    diff = te-ts
    i+=1
    prof = {"infer_time_ms":diff,"request_num":i}
    results.append(prof)

func:'model_serv_one_pred' args:[(), {}] took: 0.0332 sec
func:'model_serv_one_pred' args:[(), {}] took: 0.0326 sec
func:'model_serv_one_pred' args:[(), {}] took: 0.0316 sec
func:'model_serv_one_pred' args:[(), {}] took: 0.0313 sec
func:'model_serv_one_pred' args:[(), {}] took: 0.0322 sec
func:'model_serv_one_pred' args:[(), {}] took: 0.0313 sec
func:'model_serv_one_pred' args:[(), {}] took: 0.0314 sec
func:'model_serv_one_pred' args:[(), {}] took: 0.0312 sec
func:'model_serv_one_pred' args:[(), {}] took: 0.0320 sec
func:'model_serv_one_pred' args:[(), {}] took: 0.0313 sec
func:'model_serv_one_pred' args:[(), {}] took: 0.0313 sec
func:'model_serv_one_pred' args:[(), {}] took: 0.0321 sec
func:'model_serv_one_pred' args:[(), {}] took: 0.0315 sec
func:'model_serv_one_pred' args:[(), {}] took: 0.0316 sec
func:'model_serv_one_pred' args:[(), {}] took: 0.0327 sec
func:'model_serv_one_pred' args:[(), {}] took: 0.0320 sec
func:'model_serv_one_pred' args:[(), {}] took: 0.0317 sec
func:'model_se

In [124]:
[x.shape for x in encoded_input.values()]

[(1, 9), (1, 9), (1, 9)]

In [126]:
encoded_input.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [125]:
model_service_metadata()

{'model_spec': {'name': 'bert_base_uncased',
  'signature_name': '',
  'version': '1'},
 'metadata': {'signature_def': {'signature_def': {'serving_default': {'inputs': {'token_type_ids': {'dtype': 'DT_INT64',
       'tensor_shape': {'dim': [{'size': '-1', 'name': ''},
         {'size': '-1', 'name': ''}],
        'unknown_rank': False},
       'name': 'serving_default_token_type_ids:0'},
      'attention_mask': {'dtype': 'DT_INT64',
       'tensor_shape': {'dim': [{'size': '-1', 'name': ''},
         {'size': '-1', 'name': ''}],
        'unknown_rank': False},
       'name': 'serving_default_attention_mask:0'},
      'input_ids': {'dtype': 'DT_INT64',
       'tensor_shape': {'dim': [{'size': '-1', 'name': ''},
         {'size': '-1', 'name': ''}],
        'unknown_rank': False},
       'name': 'serving_default_input_ids:0'}},
     'outputs': {'last_hidden_state': {'dtype': 'DT_FLOAT',
       'tensor_shape': {'dim': [{'size': '-1', 'name': ''},
         {'size': '-1', 'name': ''},
     